In [1]:
from brax import envs
from prefacc.training.agents.prefppo import train as prefppo

In [5]:
env = envs.get_environment('ant', backend='positional')

In [6]:
# ant
make_inference_fn, params, metrics = prefppo.train(
  env,
  num_timesteps=50_000_000, 
  num_evals=20, 
  reward_scaling=10, 
  episode_length=1000, 
  normalize_observations=True, 
  action_repeat=1, 
  unroll_length=5, 
  num_minibatches=32, 
  num_updates_per_batch=4, 
  discounting=0.97, 
  learning_rate=3e-4, 
  entropy_cost=1e-2, 
  num_envs=4096, 
  batch_size=2048, 
  seed=1,
  num_prefs=10000)

# halfcheetah
# make_inference_fn, params, metrics = prefppo.train(
#   env,
#   num_timesteps=20_000_000, num_evals=10, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=4096, batch_size=2048, seed=1,
#   num_prefs=2000)  

# hopper
# make_inference_fn, params, metrics = prefppo.train(
#     env,
#     num_timesteps=50_000_000, num_evals=10, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=4096, batch_size=2048, seed=7438279218,
#     num_prefs=4000)  

# walker2d
# make_inference_fn, params, metrics = prefppo.train(
#   env,
#   num_timesteps=15_000_000, 
#   num_evals=15, 
#   reward_scaling=10, 
#   episode_length=1000, 
#   normalize_observations=True, 
#   action_repeat=1, 
#   unroll_length=5, 
#   num_minibatches=32, 
#   num_updates_per_batch=4, 
#   discounting=0.99, 
#   learning_rate=3e-4, 
#   entropy_cost=1e-2, 
#   num_envs=4096, 
#   batch_size=2048, 
#   seed=1)

# pusher
# make_inference_fn, params, metrics = prefppo.train(
#         env,
#         num_timesteps=50_000_000, num_evals=20, reward_scaling=5, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=30, num_minibatches=16, num_updates_per_batch=8, discounting=0.95, learning_rate=3e-4,entropy_cost=1e-2, num_envs=2048, batch_size=512, seed=3, num_prefs=10000
#     )


# pendulum
# make_inference_fn, params, metrics = prefppo.train(
#         env,
#         num_timesteps=5_000_000, num_evals=20, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=2048, batch_size=1024, seed=1, num_prefs=10000
#     )

# double_pendulum
# make_inference_fn, params, metrics = prefppo.train(
#         env,
#         # num_timesteps=50_000_000, num_evals=20, reward_scaling=5, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=30, num_minibatches=16, num_updates_per_batch=8, discounting=0.95, learning_rate=3e-4,entropy_cost=1e-2, num_envs=2048, batch_size=512, seed=3, num_prefs=10000

#         num_timesteps=30_000_000, num_evals=20, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=2048, batch_size=1024, seed=1, num_prefs=2000
#     )


print("Training complete")

Training complete


In [7]:
print(metrics)

{'eval/walltime': 44.6544075012207, 'training/sps': np.float64(538714.7004731123), 'training/walltime': 115.42767310142517, 'training/entropy_loss': Array(0.08578943, dtype=float32), 'training/policy_loss': Array(-0.00066906, dtype=float32), 'training/total_loss': Array(25.781633, dtype=float32), 'training/v_loss': Array(25.696512, dtype=float32), 'eval/episode_distance_from_origin': Array(103820.06, dtype=float32), 'eval/episode_forward_reward': Array(4245.234, dtype=float32), 'eval/episode_reward': Array(3557.9727, dtype=float32), 'eval/episode_reward_contact': Array(0., dtype=float32), 'eval/episode_reward_ctrl': Array(-1453.8942, dtype=float32), 'eval/episode_reward_forward': Array(4245.234, dtype=float32), 'eval/episode_reward_survive': Array(766.6328, dtype=float32), 'eval/episode_x_position': Array(102773.03, dtype=float32), 'eval/episode_x_velocity': Array(4245.234, dtype=float32), 'eval/episode_y_position': Array(14000.822, dtype=float32), 'eval/episode_y_velocity': Array(579.

In [8]:
from brax.io import model
from brax.io import json
from brax.io import html

In [9]:
model.save_params('/tmp/params', params)

In [10]:
params = model.load_params('/tmp/params')
inference_fn = make_inference_fn(params)

In [11]:
env = envs.create('ant', backend='positional')

In [12]:
import jax

In [13]:
jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.jit(inference_fn)

In [14]:
rollout = []
rng = jax.random.PRNGKey(seed=4231)
state = jit_env_reset(rng=rng)
for _ in range(1000):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  act, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_env_step(state, act)

In [15]:
from IPython.display import HTML, clear_output

In [16]:
HTML(html.render(env.sys.tree_replace({'opt.timestep': env.dt}), rollout))